# County Election Data

## Helper Functions, Variables and Libraries

In [1]:
from bs4 import BeautifulSoup as bs
import sys
import requests
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Thanks to Politico.com for the election results data!
data_url = "https://www.politico.com/2020-election/results/"
data_df = pd.DataFrame(columns=['County', 'GOP Votes', 'DFL Votes', 'State'])
entry_df = pd.DataFrame(columns=['County', 'GOP Votes', 'DFL Votes', 'State'])

states = ["alabama", "alaska", "arizona", "arkansas", "california",
    "colorado", "connecticut", "delaware", "florida", "georgia", "hawaii",
    "idaho", "illinois", "indiana", "iowa", "kansas", "kentucky",
    "louisiana", "maine", "maryland", "massachusetts", "michigan",
    "minnesota", "mississippi", "missouri", "montana", "nebraska", "nevada",
    "new-hampshire", "new-jersey", "new-mexico", "new-york", "north-carolina",
    "north-dakota", "ohio", "oklahoma", "oregon", "pennsylvania",
    "rhode-island", "south-carolina", "south-dakota", "tennessee", "texas",
    "utah", "vermont", "virginia", "washington", "west-virginia", "wisconsin",
    "wyoming", "washington-dc"
]

county_gop_vote_count = 0
county_dem_vote_count = 0

def click_button(button_lst):
    click_success = False
    Attempt_Cap = 0
    
    while (click_success == False and Attempt_Cap < 1000):
        Attempt_Cap += 1

        for i in range(0, len(button_lst)):
            try:
                button_test[len(button_test) - 1 - i].click()
                click_success = True
                break
            except:
                click_success = False
            
    # Now check to see if we got it!
    if (Attempt_Cap > 1000 and click_success == False):
        print("Warning!  The button click count was exceeded!")

def get_results(table_row):
    if (table_row.find('td', class_='jsx-3469064484 votes-cell dem dem-background') == None):
        return int(table_row.find('td', class_='jsx-3469064484 votes-cell gop gop-background').text.replace(",", "")), int(table_row.find('td', class_='jsx-3469064484 votes-cell dem').text.replace(",", ""))
    else:
        return int(table_row.find('td', class_='jsx-3469064484 votes-cell gop').text.replace(",", "")), int(table_row.find('td', class_='jsx-3469064484 votes-cell dem dem-background').text.replace(",", ""))

## Scraping Policito.com's tables

In [2]:
data_df = pd.DataFrame(columns=['County', 'GOP Votes', 'DFL Votes', 'State'])
#jsx-3713440361
for state in states:
    # First get the site's page in the browser
    state_page_url = data_url + state + "/"

    try:
        driver.get(state_page_url)
    except:
        # In case the browser gets closed after accessing a state's web page
        # or the initial starting of the state search loop
        driver = webdriver.Chrome(executable_path='chrome-driver/chromedriver.exe')
        driver.get(state_page_url)

    # Determine if we have a button to click to expand the county table
    time.sleep(10)
    button_test = driver.find_elements(By.CLASS_NAME, 'jsx-3713440361')

    if button_test != []:
        click_button(button_test)

    # Now parse the page for the table
    state_soup = bs(driver.page_source, 'html.parser')
    state_county_results_table = state_soup.find('tbody', class_='jsx-3713440361')

    # Now Get the full county list up on the page
    print(state.replace("-", " ").title())

    if (state_county_results_table is None):
        vote_counts = driver.find_elements(By.CLASS_NAME, 'jsx-3768461732')

        if (driver.find_elements(By.CLASS_NAME, 'jsx-1912693590')[0].text == 'Donald Trump*'):
            # Trump won this state
            gop_vote_count = vote_counts[0].text
            dem_vote_count = vote_counts[1].text
        else:
            # Biden won this state
            dem_vote_count = vote_counts[0].text
            gop_vote_count = vote_counts[1].text

        print("State Only")
        entry_df = pd.DataFrame([[None, gop_vote_count, dem_vote_count, state.replace("-", " ").title()]], columns=['County', 'GOP Votes', 'DFL Votes', 'State'])
        data_df = data_df.append(entry_df)
    else:
        print("County Count: {0}".format(len(state_county_results_table.find_all('tr'))))
        for county_row in state_county_results_table.find_all('tr'):
            county_name = county_row.find('td', class_='jsx-1352427234 county-name is-desktop').text
            county_gop_vote_count, county_dem_vote_count = get_results(county_row)

            entry_df = pd.DataFrame([[county_name, county_gop_vote_count, county_dem_vote_count, state.replace("-", " ").title()]], columns=['County', 'GOP Votes', 'DFL Votes', 'State'])
            data_df = data_df.append(entry_df)
    print('============================')

driver.close()

Alabama
County Count: 67
Alaska
State Only
Arizona
County Count: 15
Arkansas
County Count: 75
California
County Count: 58
Colorado
County Count: 64
Connecticut
County Count: 8
Delaware
County Count: 3
Florida
County Count: 67
Georgia
County Count: 159
Hawaii
County Count: 4
Idaho
County Count: 44
Illinois
County Count: 102
Indiana
County Count: 92
Iowa
County Count: 99
Kansas
County Count: 105
Kentucky
County Count: 120
Louisiana
County Count: 64
Maine
County Count: 16
Maryland
County Count: 24
Massachusetts
County Count: 14
Michigan
County Count: 83
Minnesota
County Count: 87
Mississippi
County Count: 82
Missouri
County Count: 115
Montana
County Count: 56
Nebraska
County Count: 93
Nevada
County Count: 17
New Hampshire
County Count: 10
New Jersey
County Count: 21
New Mexico
County Count: 33
New York
County Count: 62
North Carolina
County Count: 100
North Dakota
County Count: 53
Ohio
County Count: 88
Oklahoma
County Count: 77
Oregon
County Count: 36
Pennsylvania
County Count: 67
Rhode I

In [3]:
print(data_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3113 entries, 0 to 0
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   County     3111 non-null   object
 1   GOP Votes  3113 non-null   object
 2   DFL Votes  3113 non-null   object
 3   State      3113 non-null   object
dtypes: object(4)
memory usage: 121.6+ KB
None


### Now save your content!

In [4]:
data_df.to_csv('Data/CountyElectionResults.csv', index=False)
data_df.groupby(['State', 'County']).max()

GOP Votes  DFL Votes
State   County                                 
Alabama Autauga County         19838       7503
        Baldwin County         83544      24578
        Barbour County          5622       4816
        Bibb County             7525       1986
        Blount County          24711       2640
...                              ...        ...
Wyoming Sweetwater County      12229       3823
        Teton County            4341       9848
        Uinta County            7496       1591
        Washakie County         3245        651
        Weston County           3107        360

[3111 rows x 2 columns]